In [ ]:
import pandas as pd
import numpy as np

def calculate_psi_continuous(df1, df2, column, bins=10, fill_value=1):
    """
    Calculate the Population Stability Index (PSI) for a continuous column using binning based on the first DataFrame.
    
    Parameters:
    - df1: First pandas DataFrame representing the first period.
    - df2: Second pandas DataFrame representing the second period.
    - column: The name of the continuous column to analyze.
    - bins: Number of bins to divide the data (default is 10).
    - fill_value: The value to replace zero counts with (default is 1).

    Returns:
    - psi_total: The total PSI value.
    - psi_df: A pandas DataFrame showing the bin ranges, count, percentage, and PSI contribution for each bin.
    """
    
    # Create bin edges based on the first DataFrame only (reference period)
    bin_edges = np.histogram_bin_edges(df1[column], bins=bins)
    
    # Count occurrences in each bin for both periods
    df1_counts, _ = np.histogram(df1[column], bins=bin_edges)
    df2_counts, _ = np.histogram(df2[column], bins=bin_edges)
    
    # Convert counts to percentages (relative frequencies)
    df1_pct = df1_counts / len(df1)
    df2_pct = df2_counts / len(df2)
    
    # Replace zero entries with the specified fill_value
    df1_pct = np.where(df1_pct == 0, fill_value, df1_pct)
    df2_pct = np.where(df2_pct == 0, fill_value, df2_pct)
    
    # Calculate the PSI contribution for each bin
    psi_contributions = (df2_pct - df1_pct) * np.log(df2_pct / df1_pct)
    
    # Create a DataFrame with detailed PSI information for each bin
    psi_df = pd.DataFrame({
        'Bin_start': bin_edges[:-1],
        'Bin_end': bin_edges[1:],
        'Period1_count': df1_counts,
        'Period1_pct': df1_pct,
        'Period2_count': df2_counts,
        'Period2_pct': df2_pct,
        'PSI_contribution': psi_contributions
    })
    
    # Calculate the total PSI
    psi_total = np.sum(psi_contributions)
    
    return psi_total, psi_df


In [ ]:
import pandas as pd
import numpy as np

def calculate_psi_categorical(df1, df2, column, fill_value=1):
    """
    Calculate the Population Stability Index (PSI) for a categorical column.
    
    Parameters:
    - df1: First pandas DataFrame representing the first period.
    - df2: Second pandas DataFrame representing the second period.
    - column: The name of the categorical column to analyze.
    - fill_value: The value to replace zero counts with (default is 1).

    Returns:
    - psi_total: The total PSI value.
    - psi_df: A pandas DataFrame showing the count, percentage, and PSI contribution for each category.
    """
    
    # Count the occurrences for each category in both periods
    df1_counts = df1[column].value_counts(normalize=False)
    df2_counts = df2[column].value_counts(normalize=False)
    
    # Convert counts to percentages (relative frequencies)
    df1_pct = df1_counts / len(df1)
    df2_pct = df2_counts / len(df2)
    
    # Ensure all categories are present in both periods
    all_categories = pd.Index(df1_counts.index).union(df2_counts.index)
    df1_pct = df1_pct.reindex(all_categories, fill_value=0)
    df2_pct = df2_pct.reindex(all_categories, fill_value=0)
    
    # Replace zero entries with the specified fill_value
    df1_pct = np.where(df1_pct == 0, fill_value, df1_pct)
    df2_pct = np.where(df2_pct == 0, fill_value, df2_pct)
    
    # Calculate the PSI contribution for each category
    psi_contributions = (df2_pct - df1_pct) * np.log(df2_pct / df1_pct)
    
    # Create a DataFrame with detailed PSI information for each category
    psi_df = pd.DataFrame({
        'Category': all_categories,
        'Period1_count': df1_counts.reindex(all_categories, fill_value=0),
        'Period1_pct': df1_pct,
        'Period2_count': df2_counts.reindex(all_categories, fill_value=0),
        'Period2_pct': df2_pct,
        'PSI_contribution': psi_contributions
    })
    
    # Calculate the total PSI
    psi_total = np.sum(psi_contributions)
    
    return psi_total, psi_df


In [ ]:
import pandas as pd
import numpy as np

def calculate_hhi(df, column):
    """
    Calculate the Herfindahl-Hirschman Index (HHI) for a categorical column to assess concentration.
    
    Parameters:
    - df: pandas DataFrame containing the data.
    - column: The name of the categorical column to analyze.

    Returns:
    - hhi_value: The Herfindahl-Hirschman Index (HHI) value.
    - hhi_df: A pandas DataFrame showing the count, percentage, and squared percentage for each category.
    """
    
    # Count the occurrences for each category
    counts = df[column].value_counts(normalize=False)
    
    # Convert counts to percentages (relative frequencies)
    pct = counts / len(df)
    
    # Calculate the squared percentage (used in HHI)
    squared_pct = pct ** 2
    
    # Calculate the HHI as the sum of squared percentages
    hhi_value = np.sum(squared_pct)
    
    # Create a DataFrame with detailed HHI information for each category
    hhi_df = pd.DataFrame({
        'Category': counts.index,
        'Count': counts.values,
        'Percentage': pct.values,
        'Squared_Percentage': squared_pct.values
    })
    
    return hhi_value, hhi_df

# Example usage:
# hhi_value, hhi_detail_df = calculate_hhi(df, 'city_column')


In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ks_2samp, chi2_contingency
from sklearn.metrics import mean_absolute_error, mean_squared_error

# 1. Kolmogorov-Smirnov Test (KS)
def kolmogorov_smirnov_test(df1, df2, column):
    """
    Perform the Kolmogorov-Smirnov test to compare the distribution of values in two DataFrames.

    Parameters:
    df1 (pd.DataFrame): First DataFrame.
    df2 (pd.DataFrame): Second DataFrame.
    column (str): Column name on which to perform the test.

    Returns:
    ks_stat (float): The KS test statistic.
    ks_p_value (float): The p-value from the KS test.
    """
    ks_stat, ks_p_value = ks_2samp(df1[column], df2[column])
    return ks_stat, ks_p_value

# 2. Chi-Square Test
def chi_square_test(df1, df2, column):
    """
    Perform the Chi-Square test to compare the categorical distributions between two DataFrames.

    Parameters:
    df1 (pd.DataFrame): First DataFrame.
    df2 (pd.DataFrame): Second DataFrame.
    column (str): Column name with categorical data.

    Returns:
    chi2_stat (float): The Chi-Square statistic.
    p_val (float): The p-value from the Chi-Square test.
    """
    cont_table = pd.crosstab(df1[column], df2[column])
    chi2_stat, p_val, _, _ = chi2_contingency(cont_table)
    return chi2_stat, p_val

# 3. Mean Absolute Error (MAE)
def calculate_mae(df1, df2, column):
    """
    Calculate the Mean Absolute Error between two sets of values from the DataFrames.

    Parameters:
    df1 (pd.DataFrame): First DataFrame.
    df2 (pd.DataFrame): Second DataFrame.
    column (str): Column name with numeric data.

    Returns:
    mae (float): Mean Absolute Error.
    """
    return mean_absolute_error(df1[column], df2[column])

# 4. Root Mean Squared Error (RMSE)
def calculate_rmse(df1, df2, column):
    """
    Calculate the Root Mean Squared Error between two sets of values from the DataFrames.

    Parameters:
    df1 (pd.DataFrame): First DataFrame.
    df2 (pd.DataFrame): Second DataFrame.
    column (str): Column name with numeric data.

    Returns:
    rmse (float): Root Mean Squared Error.
    """
    return np.sqrt(mean_squared_error(df1[column], df2[column]))

# 5. Transition Matrix considering missing categories
def transition_matrix(df1, df2, category_column):
    """
    Create a transition matrix showing how categories change between two time periods, considering missing categories.

    Parameters:
    df1 (pd.DataFrame): First DataFrame containing 'asset_id' and category_column.
    df2 (pd.DataFrame): Second DataFrame containing 'asset_id' and category_column.
    category_column (str): Column name containing the categorical data.

    Returns:
    transition_matrix (pd.DataFrame): A DataFrame representing the transition matrix, normalized by row.
    """
    merged_df = pd.merge(df1[['asset_id', category_column]], 
                         df2[['asset_id', category_column]], 
                         on='asset_id', 
                         suffixes=('_t1', '_t2'),
                         how='outer')
    
    # Replace missing values with 'Missing' to account for entries that do not exist in both DataFrames
    merged_df.fillna('Missing', inplace=True)

    # Create transition matrix normalized by row
    transition_matrix = pd.crosstab(merged_df[category_column + '_t1'], 
                                    merged_df[category_column + '_t2'], 
                                    normalize='index')
    return transition_matrix

# 6. Percentage transition matrix
def percentage_transition(transition_matrix):
    """
    Calculate the percentage representation of each transition in the transition matrix.

    Parameters:
    transition_matrix (pd.DataFrame): A transition matrix DataFrame.

    Returns:
    transition_percentages (pd.DataFrame): A DataFrame representing transition percentages per row.
    """
    transition_percentages = transition_matrix.apply(lambda x: x / x.sum(), axis=1)
    return transition_percentages

# --- Example usage of the functions ---

# Assuming df1 and df2 are your DataFrames and 'value' and 'category' are the respective columns
ks_stat, ks_p_value = kolmogorov_smirnov_test(df1, df2, 'value')
chi2_stat, chi2_p_value = chi_square_test(df1, df2, 'category')
mae = calculate_mae(df1, df2, 'value')
rmse = calculate_rmse(df1, df2, 'value')

# Create transition matrix and calculate percentages
trans_matrix = transition_matrix(df1, df2, 'category')
transition_percentages = percentage_transition(trans_matrix)

# Output results
print(f"Kolmogorov-Smirnov test: Statistic={ks_stat}, p-value={ks_p_value}")
print(f"Chi-Square test: Statistic={chi2_stat}, p-value={chi2_p_value}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print("Transition Matrix:")
print(trans_matrix)
print("Transition Percentages per row:")
print(transition_percentages)


C:\Users\gonza\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


NameError: name 'df1' is not defined

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ks_2samp

# 1. Kolmogorov-Smirnov Distance Plot (CDFs) with flexible plot size
def plot_ks_distance(df1, df2, column, figsize=(10, 6)):
    """
    Plot the CDFs of two datasets and show the Kolmogorov-Smirnov distance.
    
    Parameters:
    df1 (pd.DataFrame): First DataFrame.
    df2 (pd.DataFrame): Second DataFrame.
    column (str): Column name with numerical data to compare.
    figsize (tuple): Size of the plot (default is (10, 6)).
    
    Returns:
    None: Displays the plot.
    """
    # Sort values
    data1_sorted = np.sort(df1[column])
    data2_sorted = np.sort(df2[column])

    # CDFs
    cdf1 = np.arange(1, len(data1_sorted)+1) / len(data1_sorted)
    cdf2 = np.arange(1, len(data2_sorted)+1) / len(data2_sorted)

    # Plot CDFs
    plt.figure(figsize=figsize)
    plt.step(data1_sorted, cdf1, label='Initial Data', color='blue')
    plt.step(data2_sorted, cdf2, label='Future Data', color='green')
    plt.title(f'Kolmogorov-Smirnov Distance for {column}')
    plt.xlabel(f'{column}')
    plt.ylabel('CDF')
    plt.legend(loc='best')

    # Kolmogorov-Smirnov distance
    ks_stat, _ = ks_2samp(df1[column], df2[column])
    plt.text(0.1, 0.9, f'KS Distance = {ks_stat:.4f}', transform=plt.gca().transAxes, fontsize=12)

    plt.show()

# 2. Histograms or KDEs with customizable bins and plot size
def plot_distribution_comparison(df1, df2, column, plot_type='hist', bins=30, figsize=(10, 6)):
    """
    Plot a comparison of distributions using either histograms or KDEs.

    Parameters:
    df1 (pd.DataFrame): First DataFrame.
    df2 (pd.DataFrame): Second DataFrame.
    column (str): Column name with numerical data.
    plot_type (str): Type of plot ('hist' for histogram, 'kde' for Kernel Density Estimate).
    bins (int): Number of bins for histogram (default is 30).
    figsize (tuple): Size of the plot (default is (10, 6)).

    Returns:
    None: Displays the plot.
    """
    plt.figure(figsize=figsize)

    if plot_type == 'hist':
        plt.hist(df1[column], bins=bins, alpha=0.5, label='Initial Data', color='blue', density=True)
        plt.hist(df2[column], bins=bins, alpha=0.5, label='Future Data', color='green', density=True)
    elif plot_type == 'kde':
        df1[column].plot(kind='kde', label='Initial Data', color='blue', linewidth=2)
        df2[column].plot(kind='kde', label='Future Data', color='green', linewidth=2)

    plt.title(f'Distribution Comparison for {column}')
    plt.xlabel(f'{column}')
    plt.ylabel('Density')
    plt.legend(loc='best')

    plt.show()

# 3. Boxplot comparison with customizable plot size
def plot_boxplot_comparison(df1, df2, column, figsize=(10, 6)):
    """
    Plot a boxplot comparison for a numerical column across two datasets.

    Parameters:
    df1 (pd.DataFrame): First DataFrame.
    df2 (pd.DataFrame): Second DataFrame.
    column (str): Column name with numerical data.
    figsize (tuple): Size of the plot (default is (10, 6)).

    Returns:
    None: Displays the boxplot.
    """
    plt.figure(figsize=figsize)
    
    data = [df1[column], df2[column]]
    plt.boxplot(data, labels=['Initial Data', 'Future Data'], patch_artist=True,
                boxprops=dict(facecolor='lightblue'), medianprops=dict(color='red'))
    
    plt.title(f'Boxplot Comparison for {column}')
    plt.ylabel(f'{column}')
    
    plt.show()


In [3]:
def plot_categorical_comparison(df1, df2, column, figsize=(10, 6)):
    """
    Plot a comparison of categorical data proportions between two datasets.

    Parameters:
    df1 (pd.DataFrame): First DataFrame.
    df2 (pd.DataFrame): Second DataFrame.
    column (str): Column name with categorical data.
    figsize (tuple): Size of the plot (default is (10, 6)).

    Returns:
    None: Displays the bar plot.
    """
    # Calculate proportions in both DataFrames
    prop1 = df1[column].value_counts(normalize=True)
    prop2 = df2[column].value_counts(normalize=True)

    # Create a DataFrame to hold both proportions
    comparison_df = pd.DataFrame({'Initial Data': prop1, 'Future Data': prop2}).fillna(0)

    # Plot the comparison
    comparison_df.plot(kind='bar', figsize=figsize)
    plt.title(f'Categorical Comparison for {column}')
    plt.xlabel(f'{column}')
    plt.ylabel('Proportion')
    plt.xticks(rotation=45)
    plt.legend(loc='best')

    plt.show()

# Example usage
# Assuming df1 and df2 are your DataFrames and 'city' is the categorical column you want to compare
plot_ks_distance(df1, df2, 'price', figsize=(12, 8))  # Custom plot size for KS Distance
plot_distribution_comparison(df1, df2, 'price', plot_type='hist', bins=40)  # Custom bins for histogram
plot_boxplot_comparison(df1, df2, 'price', figsize=(8, 6))  # Custom plot size for boxplot
plot_categorical_comparison(df1, df2, 'city', figsize=(12, 6))  # Categorical comparison plot


NameError: name 'df1' is not defined

In [ ]:
import pandas as pd

def rename_and_validate_dataframes(df1, df2, column_mapping_df1, column_mapping_df2, dtype_mapping):
    """
    Renames columns in two DataFrames based on provided dictionaries and checks if the column data types match
    the expected types. If the data types don't match, it attempts to cast the columns and raises a flag if unable.
    The original column is kept if casting fails, and a message is printed indicating which columns need to be reviewed.

    Parameters:
    df1 (pd.DataFrame): First DataFrame to be processed.
    df2 (pd.DataFrame): Second DataFrame to be processed.
    column_mapping_df1 (dict): Dictionary with keys as current column names in df1 and values as consolidated column names.
    column_mapping_df2 (dict): Dictionary with keys as current column names in df2 and values as consolidated column names.
    dtype_mapping (dict): Dictionary specifying the expected data type for each consolidated column name.

    Returns:
    df1_renamed (pd.DataFrame): First DataFrame with renamed columns and validated types.
    df2_renamed (pd.DataFrame): Second DataFrame with renamed columns and validated types.
    invalid_columns (list): List of columns that could not be casted to the expected data type.
    """
    # Rename columns based on separate mappings for each DataFrame
    df1_renamed = df1.rename(columns=column_mapping_df1)
    df2_renamed = df2.rename(columns=column_mapping_df2)
    
    # List to track columns that could not be cast
    invalid_columns = []

    # Iterate over dtype_mapping to check and cast types
    for column, expected_dtype in dtype_mapping.items():
        if column in df1_renamed.columns:
            # Check if df1 column can be cast to the expected dtype
            if not pd.api.types.is_dtype_equal(df1_renamed[column].dtype, expected_dtype):
                try:
                    df1_renamed[column] = df1_renamed[column].astype(expected_dtype)
                except (ValueError, TypeError):
                    # If casting fails, keep the original column and flag it
                    invalid_columns.append((column, 'df1', df1_renamed[column].dtype))
                    print(f"Column '{column}' in df1 has type {df1_renamed[column].dtype} and couldn't be cast to {expected_dtype}.")
        
        if column in df2_renamed.columns:
            # Check if df2 column can be cast to the expected dtype
            if not pd.api.types.is_dtype_equal(df2_renamed[column].dtype, expected_dtype):
                try:
                    df2_renamed[column] = df2_renamed[column].astype(expected_dtype)
                except (ValueError, TypeError):
                    # If casting fails, keep the original column and flag it
                    invalid_columns.append((column, 'df2', df2_renamed[column].dtype))
                    print(f"Column '{column}' in df2 has type {df2_renamed[column].dtype} and couldn't be cast to {expected_dtype}.")

    return df1_renamed, df2_renamed, invalid_columns

# Example usage
# Assuming df1 and df2 are the two DataFrames you want to process
# column_mapping_df1 maps current column names in df1 to the desired consolidated names
column_mapping_df1 = {
    'asset_price': 'price',
    'asset_area': 'area',
    'city_name': 'city',
    # Add more mappings as needed
}

# column_mapping_df2 maps current column names in df2 to the desired consolidated names
column_mapping_df2 = {
    'price_asset': 'price',
    'area_asset': 'area',
    'location_city': 'city',
    # Add more mappings as needed
}

# dtype_mapping specifies the expected data type for each consolidated column
dtype_mapping = {
    'price': 'float64',
    'area': 'float64',
    'city': 'object',  # 'object' is the dtype for strings in pandas
    # Add more expected types as needed
}

df1_renamed, df2_renamed, invalid_columns = rename_and_validate_dataframes(df1, df2, column_mapping_df1, column_mapping_df2, dtype_mapping)

# Output results
if invalid_columns:
    print("The following columns could not be cast to the expected data type and were left as is:")
    for col, df_name, current_dtype in invalid_columns:
        print(f"Column '{col}' in {df_name} has type {current_dtype} and couldn't be cast to {dtype_mapping[col]}.")
else:
    print("All columns have been successfully cast to the expected data types.")
